In [ ]:
"""
# 🎯 Notebook 05 : Recommandation d'Offres d'Emploi

**Objectif** : Matcher un CV avec toutes les offres et recommander les plus pertinentes

**Pipeline** :
1. Charger le CV et ses compétences extraites (Étape 2)
2. Charger toutes les offres d'emploi (Étape 4)
3. Utiliser le JobMatcher pour calculer les scores
4. Trier et recommander les TOP offres

**Modules utilisés** :
- `src/cv_parser.py` : Parser de CV
- `src/job_matcher.py` : Système de matching sémantique
"""

In [14]:
# Imports
import sys
import json
from pathlib import Path
import numpy as np
from typing import List, Dict

# Ajouter le dossier racine
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

print(f"📁 Projet : {project_root}")
print(f"📂 Notebooks : {Path().absolute()}")

# Importer nos modules custom
try:
    from src.job_matcher import JobMatcher, load_matcher
    print("✅ JobMatcher importé depuis src/job_matcher.py")
except ImportError as e:
    print(f"❌ Erreur d'import : {e}")
    print("⚠️  Vérifiez que src/job_matcher.py existe")

print("\n✅ Imports terminés")

📁 Projet : c:\Users\rober\OneDrive\Bureau\PFE
📂 Notebooks : c:\Users\rober\OneDrive\Bureau\PFE\notebooks
✅ JobMatcher importé depuis src/job_matcher.py

✅ Imports terminés


In [15]:
# Charger les compétences extraites du CV (Étape 2)
skills_path = project_root / "outputs" / "extracted_skills_simple.json"

if not skills_path.exists():
    print("❌ Fichier compétences non trouvé")
    print(f"⚠️  Chemin attendu : {skills_path}")
    print("⚠️  Exécutez d'abord 02_skills_extraction_simple.ipynb")
    cv_skills = []
else:
    with open(skills_path, 'r', encoding='utf-8') as f:
        cv_data = json.load(f)
    
    cv_skills = cv_data['technical_skills']
    cv_soft_skills = cv_data.get('soft_skills', [])
    
    print(f"✅ Compétences CV chargées")
    print(f"   • Fichier : {cv_data.get('cv_file', 'N/A')}")
    print(f"   • Date extraction : {cv_data.get('extraction_date', 'N/A')}")
    print(f"   • Compétences techniques : {len(cv_skills)}")
    print(f"   • Soft skills : {len(cv_soft_skills)}")
    print(f"\n📋 Exemples de compétences :")
    for skill in cv_skills[:10]:
        print(f"   • {skill}")

✅ Compétences CV chargées
   • Fichier : RESUME ROBERT UNG.pdf
   • Date extraction : 2026-01-25
   • Compétences techniques : 24
   • Soft skills : 5

📋 Exemples de compétences :
   • .net
   • artificial intelligence
   • big data
   • c
   • c#
   • c++
   • data analysis
   • data science
   • deep learning
   • docker


In [16]:
# Charger toutes les offres d'emploi (Étape 4)
jobs_dir = project_root / "data" / "jobs"
all_jobs = []

if not jobs_dir.exists():
    print(f"❌ Dossier jobs/ non trouvé : {jobs_dir}")
    print("⚠️  Exécutez d'abord 04_job_generation.ipynb")
else:
    # Charger le dataset complet
    dataset_path = jobs_dir / "jobs_dataset.json"
    
    if dataset_path.exists():
        with open(dataset_path, 'r', encoding='utf-8') as f:
            dataset = json.load(f)
        
        all_jobs = dataset['jobs']
        metadata = dataset['metadata']
        
        print(f"✅ Offres d'emploi chargées")
        print(f"   • Fichier : {dataset_path.name}")
        print(f"   • Total offres : {metadata['total_jobs']}")
        print(f"   • Généré le : {metadata['generated_date']}")
        print(f"   • Catégories : {', '.join(metadata['categories'])}")
        
        # Statistiques par catégorie
        print(f"\n📊 Répartition par catégorie :")
        for category in metadata['categories']:
            count = len([j for j in all_jobs if j['category'] == category])
            print(f"   • {category:20s} : {count} offres")
    else:
        print(f"❌ Fichier {dataset_path.name} non trouvé")
        print("⚠️  Exécutez 04_job_generation.ipynb")

✅ Offres d'emploi chargées
   • Fichier : jobs_dataset.json
   • Total offres : 25
   • Généré le : 2026-01-25 18:21:43
   • Catégories : ml_engineer, data_scientist, python_developer, devops_engineer, frontend_developer

📊 Répartition par catégorie :
   • ml_engineer          : 5 offres
   • data_scientist       : 5 offres
   • python_developer     : 5 offres
   • devops_engineer      : 5 offres
   • frontend_developer   : 5 offres


In [17]:
# Charger le modèle de matching sémantique
print("\n🔍 Initialisation du JobMatcher...")
print("⏳ Chargement du modèle all-mpnet-base-v2 (première fois : ~420 MB)")

try:
    matcher = load_matcher(model_name='all-mpnet-base-v2')
    print(f"✅ JobMatcher initialisé")
    print(f"   • Modèle : {matcher.model_name}")
    print(f"   • Dimension embeddings : 768")
except Exception as e:
    print(f"❌ Erreur : {e}")
    matcher = None


🔍 Initialisation du JobMatcher...
⏳ Chargement du modèle all-mpnet-base-v2 (première fois : ~420 MB)
🔍 Chargement du modèle all-mpnet-base-v2...
✅ Modèle chargé
✅ JobMatcher initialisé
   • Modèle : all-mpnet-base-v2
   • Dimension embeddings : 768


In [18]:
# Calculer le score pour chaque offre avec le JobMatcher
if not cv_skills:
    print("❌ Aucune compétence CV chargée")
elif not all_jobs:
    print("❌ Aucune offre chargée")
elif matcher is None:
    print("❌ JobMatcher non initialisé")
else:
    print("\n🔍 Calcul des scores de matching...")
    print(f"⏳ Analyse de {len(all_jobs)} offres avec {len(cv_skills)} compétences...")
    print("   (Temps estimé : 30-60 secondes)\n")
    
    # Utiliser la méthode rank_jobs du JobMatcher
    recommendations = matcher.rank_jobs(cv_skills, all_jobs)
    
    print(f"✅ Analyse terminée !")
    print(f"   • {len(recommendations)} offres scorées")
    print(f"   • Meilleur score : {recommendations[0]['global_score']:.1f}%")
    print(f"   • Score moyen : {sum(r['global_score'] for r in recommendations) / len(recommendations):.1f}%")
    print(f"   • Pire score : {recommendations[-1]['global_score']:.1f}%")


🔍 Calcul des scores de matching...
⏳ Analyse de 25 offres avec 24 compétences...
   (Temps estimé : 30-60 secondes)

✅ Analyse terminée !
   • 25 offres scorées
   • Meilleur score : 66.7%
   • Score moyen : 52.3%
   • Pire score : 40.9%


In [19]:
# Afficher les 10 meilleures recommandations
if 'recommendations' in locals() and recommendations:
    print("\n" + "="*70)
    print("🏆 TOP 10 OFFRES RECOMMANDÉES")
    print("="*70)
    
    for i, job in enumerate(recommendations[:10], 1):
        # Emoji selon le score
        if job['global_score'] >= 70:
            emoji = "🟢"
            label = "EXCELLENT MATCH"
        elif job['global_score'] >= 50:
            emoji = "🟡"
            label = "BON MATCH"
        elif job['global_score'] >= 40:
            emoji = "🟠"
            label = "MATCH MOYEN"
        else:
            emoji = "🔴"
            label = "MATCH FAIBLE"
        
        print(f"\n{emoji} #{i} - {job['title']} ({label})")
        print("-" * 70)
        print(f"   🏢 Entreprise      : {job['company']}")
        print(f"   📍 Localisation    : {job['location']} {'🏠 (Remote)' if job['remote_ok'] else ''}")
        print(f"   💼 Type            : {job['type']} | Expérience : {job['experience']}")
        print(f"   💰 Salaire         : {job['salary']}")
        print(f"   👥 Candidats       : {job['applicants']}")
        
        print(f"\n   📊 SCORES DÉTAILLÉS :")
        print(f"      • Global       : {job['global_score']:.1f}% ⭐")
        print(f"      • Compétences  : {job['skills_score']:.1f}%")
        print(f"      • Expérience   : {job['experience_score']}%")
        print(f"      • Localisation : {job['location_score']}%")
        print(f"      • Compétition  : {job['competition_score']}%")
        
        print(f"\n   🎯 TOP COMPÉTENCES MATCHÉES :")
        for skill in job['skills_details']['top_skills']:
            print(f"      ✓ {skill}")
        
        print(f"\n   🔧 Compétences requises (extrait) :")
        for req in job['requirements'][:3]:
            print(f"      • {req}")
        
        print(f"\n   🔗 Postuler : {job['url']}")


🏆 TOP 10 OFFRES RECOMMANDÉES

🟡 #1 - Data Scientist Junior (BON MATCH)
----------------------------------------------------------------------
   🏢 Entreprise      : Analytics Pro
   📍 Localisation    : Nantes, France 🏠 (Remote)
   💼 Type            : CDI | Expérience : 0-2 ans
   💰 Salaire         : 35-45K€
   👥 Candidats       : 116

   📊 SCORES DÉTAILLÉS :
      • Global       : 66.7% ⭐
      • Compétences  : 45.4%
      • Expérience   : 100%
      • Localisation : 100%
      • Compétition  : 40%

   🎯 TOP COMPÉTENCES MATCHÉES :
      ✓ scikit-learn
      ✓ matplotlib
      ✓ jupyter
      ✓ numpy
      ✓ data analysis

   🔧 Compétences requises (extrait) :
      • Python (pandas, numpy, matplotlib)
      • Machine Learning (scikit-learn)
      • SQL

   🔗 Postuler : https://example-jobs.com/6

🟡 #2 - Python Developer Junior (BON MATCH)
----------------------------------------------------------------------
   🏢 Entreprise      : WebDev Studio
   📍 Localisation    : Toulouse, France 

In [20]:
# Statistiques sur les recommandations
if 'recommendations' in locals() and recommendations:
    print("\n" + "="*70)
    print("📊 STATISTIQUES DES RECOMMANDATIONS")
    print("="*70)
    
    # Distribution des scores
    excellent_match = [j for j in recommendations if j['global_score'] >= 70]
    good_match = [j for j in recommendations if 50 <= j['global_score'] < 70]
    medium_match = [j for j in recommendations if 40 <= j['global_score'] < 50]
    low_match = [j for j in recommendations if j['global_score'] < 40]
    
    print(f"\n🎯 Distribution des matches :")
    print(f"   • Excellent match (≥70%)  : {len(excellent_match):2d} offres 🟢")
    print(f"   • Bon match (50-70%)      : {len(good_match):2d} offres 🟡")
    print(f"   • Match moyen (40-50%)    : {len(medium_match):2d} offres 🟠")
    print(f"   • Match faible (<40%)     : {len(low_match):2d} offres 🔴")
    
    # Score moyen par catégorie
    print(f"\n📂 Score moyen par catégorie :")
    categories = set(job.get('category', 'unknown') for job in all_jobs)
    
    category_scores = {}
    for category in categories:
        cat_jobs = [r for r in recommendations 
                    if any(j['job_id'] == r['job_id'] and j.get('category') == category 
                          for j in all_jobs)]
        
        if cat_jobs:
            avg_score = sum(j['global_score'] for j in cat_jobs) / len(cat_jobs)
            category_scores[category] = avg_score
    
    # Trier par score décroissant
    for category, avg_score in sorted(category_scores.items(), key=lambda x: x[1], reverse=True):
        bar_length = int(avg_score / 5)
        bar = "█" * bar_length
        print(f"   • {category:25s} : {avg_score:5.1f}% {bar}")
    
    # Remote vs On-site
    remote_jobs = [j for j in recommendations if j['remote_ok']]
    onsite_jobs = [j for j in recommendations if not j['remote_ok']]
    
    print(f"\n🏠 Remote vs On-site :")
    if remote_jobs:
        avg_remote = sum(j['global_score'] for j in remote_jobs) / len(remote_jobs)
        print(f"   • Offres remote  ({len(remote_jobs):2d}) : {avg_remote:.1f}% 🏠")
    
    if onsite_jobs:
        avg_onsite = sum(j['global_score'] for j in onsite_jobs) / len(onsite_jobs)
        print(f"   • Offres on-site ({len(onsite_jobs):2d}) : {avg_onsite:.1f}% 🏢")
    
    # Offres par niveau d'expérience
    print(f"\n💼 Recommandations par niveau :")
    exp_levels = {}
    for job in recommendations:
        exp = job['experience']
        if exp not in exp_levels:
            exp_levels[exp] = {'count': 0, 'avg_score': 0}
        exp_levels[exp]['count'] += 1
        exp_levels[exp]['avg_score'] += job['global_score']
    
    for exp, data in sorted(exp_levels.items(), key=lambda x: x[1]['avg_score'], reverse=True):
        avg = data['avg_score'] / data['count']
        print(f"   • {exp:15s} : {data['count']:2d} offres | Score moyen : {avg:.1f}%")


📊 STATISTIQUES DES RECOMMANDATIONS

🎯 Distribution des matches :
   • Excellent match (≥70%)  :  0 offres 🟢
   • Bon match (50-70%)      : 16 offres 🟡
   • Match moyen (40-50%)    :  9 offres 🟠
   • Match faible (<40%)     :  0 offres 🔴

📂 Score moyen par catégorie :
   • data_scientist            :  56.8% ███████████
   • python_developer          :  54.4% ██████████
   • ml_engineer               :  52.5% ██████████
   • devops_engineer           :  49.0% █████████
   • frontend_developer        :  48.8% █████████

🏠 Remote vs On-site :
   • Offres remote  ( 8) : 56.6% 🏠
   • Offres on-site (17) : 50.3% 🏢

💼 Recommandations par niveau :
   • 2-4 ans         :  8 offres | Score moyen : 52.6%
   • 0-2 ans         :  5 offres | Score moyen : 60.6%
   • 1-3 ans         :  5 offres | Score moyen : 53.0%
   • 3-5 ans         :  4 offres | Score moyen : 45.0%
   • 4-6 ans         :  2 offres | Score moyen : 46.0%
   • 5-7 ans         :  1 offres | Score moyen : 46.2%


In [21]:
# Analyser les compétences les plus demandées
if 'recommendations' in locals() and recommendations:
    print("\n" + "="*70)
    print("🔧 COMPÉTENCES LES PLUS DEMANDÉES (TOP offres)")
    print("="*70)
    
    # Collecter toutes les compétences des 10 meilleures offres
    from collections import Counter
    
    all_requirements = []
    for job in recommendations[:10]:
        all_requirements.extend(job['requirements'])
    
    # Compter les occurrences
    skill_counts = Counter(all_requirements)
    
    print(f"\n📋 TOP 15 compétences recherchées :")
    for skill, count in skill_counts.most_common(15):
        bar = "█" * count
        print(f"   • {skill:40s} : {bar} ({count})")
    
    # Compétences manquantes (dans les requirements mais pas dans le CV)
    print(f"\n⚠️  Compétences à développer (présentes dans les TOP offres) :")
    
    cv_skills_lower = [s.lower() for s in cv_skills]
    missing_skills = set()
    
    for job in recommendations[:10]:
        for req in job['requirements']:
            req_lower = req.lower()
            # Vérification simple (pas de matching sémantique ici)
            if not any(cv_skill in req_lower or req_lower in cv_skill 
                      for cv_skill in cv_skills_lower):
                missing_skills.add(req)
    
    if missing_skills:
        for skill in sorted(missing_skills)[:10]:
            print(f"   • {skill}")
    else:
        print("   ✅ Vous avez toutes les compétences principales !")


🔧 COMPÉTENCES LES PLUS DEMANDÉES (TOP offres)

📋 TOP 15 compétences recherchées :
   • SQL                                      : ███ (3)
   • Statistiques                             : ██ (2)
   • Git                                      : ██ (2)
   • SQL avancé                               : ██ (2)
   • Python avancé                            : ██ (2)
   • Python (pandas, numpy, matplotlib)       : █ (1)
   • Machine Learning (scikit-learn)          : █ (1)
   • Communication (présentation de résultats) : █ (1)
   • Python (bonnes pratiques)                : █ (1)
   • APIs RESTful                             : █ (1)
   • Tests unitaires (pytest)                 : █ (1)
   • Python (numpy, pandas, scikit-learn)     : █ (1)
   • Machine Learning basics (supervised learning) : █ (1)
   • Git et GitHub                            : █ (1)
   • Docker (notions de base)                 : █ (1)

⚠️  Compétences à développer (présentes dans les TOP offres) :
   • APIs RESTful
   • Airflow


In [22]:
# Sauvegarder les recommandations
if 'recommendations' in locals() and recommendations:
    output_dir = project_root / "outputs"
    output_dir.mkdir(exist_ok=True)
    
    output_path = output_dir / "job_recommendations.json"
    
    # Statistiques détaillées
    stats = {
        "excellent_match": len([j for j in recommendations if j['global_score'] >= 70]),
        "good_match": len([j for j in recommendations if 50 <= j['global_score'] < 70]),
        "medium_match": len([j for j in recommendations if 40 <= j['global_score'] < 50]),
        "low_match": len([j for j in recommendations if j['global_score'] < 40]),
        "best_score": recommendations[0]['global_score'],
        "worst_score": recommendations[-1]['global_score'],
        "average_score": sum(r['global_score'] for r in recommendations) / len(recommendations),
        "best_category": None,
        "category_scores": {}
    }
    
    # Calculer les scores par catégorie
    categories = set(job.get('category', 'unknown') for job in all_jobs)
    for category in categories:
        cat_jobs = [r for r in recommendations 
                    if any(j['job_id'] == r['job_id'] and j.get('category') == category 
                          for j in all_jobs)]
        if cat_jobs:
            avg_score = sum(j['global_score'] for j in cat_jobs) / len(cat_jobs)
            stats['category_scores'][category] = round(avg_score, 1)
    
    # Meilleure catégorie
    if stats['category_scores']:
        stats['best_category'] = max(stats['category_scores'].items(), key=lambda x: x[1])[0]
    
    save_data = {
        "cv_file": cv_data.get('cv_file', 'RESUME ROBERT UNG.pdf'),
        "analysis_date": "2026-01-25",
        "model": matcher.model_name if matcher else "all-mpnet-base-v2",
        "total_jobs_analyzed": len(all_jobs),
        "cv_skills_count": len(cv_skills),
        "top_10_recommendations": recommendations[:10],
        "all_recommendations": recommendations,
        "statistics": stats
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(save_data, f, indent=2, ensure_ascii=False)
    
    print(f"\n💾 Résultats sauvegardés : {output_path}")
    print(f"📊 Taille : {output_path.stat().st_size / 1024:.1f} KB")
    
    print("\n✅ Recommandation d'offres terminée avec succès !")
    print(f"\n📋 Résumé :")
    print(f"   • {len(recommendations)} offres analysées")
    print(f"   • TOP offre : {recommendations[0]['title']} ({recommendations[0]['global_score']:.1f}%)")
    print(f"   • Meilleure catégorie : {stats['best_category']} ({stats['category_scores'].get(stats['best_category'], 0):.1f}%)")
    print(f"\n📂 Fichiers générés :")
    print(f"   • {output_path}")


💾 Résultats sauvegardés : c:\Users\rober\OneDrive\Bureau\PFE\outputs\job_recommendations.json
📊 Taille : 45.0 KB

✅ Recommandation d'offres terminée avec succès !

📋 Résumé :
   • 25 offres analysées
   • TOP offre : Data Scientist Junior (66.7%)
   • Meilleure catégorie : data_scientist (56.8%)

📂 Fichiers générés :
   • c:\Users\rober\OneDrive\Bureau\PFE\outputs\job_recommendations.json


In [23]:
# Générer un rapport visuel simple (sans matplotlib)
if 'recommendations' in locals() and recommendations:
    print("\n" + "="*70)
    print("📈 RAPPORT VISUEL DES RECOMMANDATIONS")
    print("="*70)
    
    # Distribution des scores (histogramme ASCII)
    print(f"\n📊 Distribution des scores :")
    
    ranges = [
        (0, 40, "< 40%  🔴"),
        (40, 50, "40-50% 🟠"),
        (50, 60, "50-60% 🟡"),
        (60, 70, "60-70% 🟢"),
        (70, 100, "≥ 70%  🟢")
    ]
    
    for min_score, max_score, label in ranges:
        count = len([j for j in recommendations 
                    if min_score <= j['global_score'] < max_score])
        bar = "█" * count
        print(f"   {label} : {bar} ({count})")
    
    # Top 5 entreprises
    print(f"\n🏢 Top 5 entreprises par score moyen :")
    companies = {}
    for job in recommendations:
        company = job['company']
        if company not in companies:
            companies[company] = []
        companies[company].append(job['global_score'])
    
    company_avg = {c: sum(scores)/len(scores) for c, scores in companies.items()}
    for i, (company, avg_score) in enumerate(sorted(company_avg.items(), 
                                                     key=lambda x: x[1], 
                                                     reverse=True)[:5], 1):
        print(f"   {i}. {company:30s} : {avg_score:.1f}%")
    
    # Offres à candidater en priorité
    print(f"\n🎯 OFFRES PRIORITAIRES (score ≥ 60% et < 50 candidats) :")
    priority_jobs = [j for j in recommendations 
                     if j['global_score'] >= 60 and j['applicants'] < 50]
    
    if priority_jobs:
        for i, job in enumerate(priority_jobs[:5], 1):
            print(f"   {i}. {job['title']:40s} | Score: {job['global_score']:.1f}% | {job['applicants']} candidats")
    else:
        print("   ⚠️  Aucune offre prioritaire trouvée (critères : score ≥60%, candidats <50)")
    
    print("\n" + "="*70)
    print("✅ Analyse complète terminée !")
    print("="*70)


📈 RAPPORT VISUEL DES RECOMMANDATIONS

📊 Distribution des scores :
   < 40%  🔴 :  (0)
   40-50% 🟠 : █████████ (9)
   50-60% 🟡 : █████████████ (13)
   60-70% 🟢 : ███ (3)
   ≥ 70%  🟢 :  (0)

🏢 Top 5 entreprises par score moyen :
   1. Analytics Pro                  : 66.7%
   2. WebDev Studio                  : 64.4%
   3. AI Startup Paris               : 61.2%
   4. MarketingTech                  : 58.3%
   5. FinTech Solutions              : 58.3%

🎯 OFFRES PRIORITAIRES (score ≥ 60% et < 50 candidats) :
   ⚠️  Aucune offre prioritaire trouvée (critères : score ≥60%, candidats <50)

✅ Analyse complète terminée !
